# Project Big Data Science (May 2020)

In [1]:
import tweepy
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
import matplotlib.pyplot as plt

# Download the Dutch stop words from the NLTK repository.
nltk.download('stopwords')

#"44.4415,-102.6855,1000km"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wannesvanleemput/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Consumer keys and access tokens, used for OAuth
consumer_key = "H2oOuvgoFBQ4PA1K9Yd8CqdM6"        
consumer_secret = "zSa6ulXVdNAl1Xk6TMSw48nVXIm88suBF06JzmT5XNSG2AIBxH"    
access_token = "1245776529768034306-YDUC9vTttvxvyhDhRVxGjfbt01p3bd"          
access_token_secret = "7cDHZiGpHSD2Y8Fe6RdRIpe75WephsSmfU6woDhHlD5BX"

auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
#auth.set_access_token(access_token, access_token_secret)

# Calling the api: we want a lot of data in one run so we wait for the limit rather than run into error...
api = tweepy.API(
                auth, 
                wait_on_rate_limit=True,
                wait_on_rate_limit_notify=True
                ) 

if not api:
    print("Can't authenticate. Check if credentials are correct.")

In [3]:
"""
TODO: Best to do data mining with specific class once program becomes more complex.
"""
class DataMiner():
    pass

In [4]:
"""
Class for the preprocessing of tweets; involves removing hyperlinks and stopwords.
"""
class PreProcessTweets():
    
    def __init__(self, 
                 tweets, 
                 remove_tags=False, 
                 remove_stopwords=False, 
                 remove_urls=False,
                 remove_mentions=False,
                 remove_punctuation=False):
        
        self.tweets = tweets
        self.remove_tags = remove_tags
        self.remove_stopwords = remove_stopwords
        self.remove_urls = remove_urls
        self.remove_mentions = remove_mentions
        self.remove_punctuation = remove_punctuation
        
    def _remove_urls(self):
        """ Remove all urls from the tweet text. """
        self.tweets = [re.sub(r'\s?http\S+', "", t) for t in self.tweets]
    
    def _remove_stopwords(self):
        """ Remove English stopwords from the text. """
        sw = set(stopwords.words("english")) 
        self.tweets = [" ".join([word for word in c.split() if word not in sw]) for c in self.tweets]

    def _remove_hashtag(self, tag=None):
        """ Remove a specific hashtag. If no tag specified, remove all tags."""
        for index, tweet in enumerate(self.tweets):
            words = tweet.split()
            no_tags = [word for word in words if "#" not in word]
            self.tweets[index] = " ".join(no_tags)
            
    def _remove_mentions(self):
        """ Remove all mentions (@user). """
        self.tweets = [re.sub(r'\s?@\S+', "", t) for t in self.tweets]
    
    def _remove_punctuation(self):
        """ Punctuation affects words: eg. 'however' is not the same word as 'however,'"""
        pass
    
    def preprocess(self):
        """ Perform the requested steps of the preprocessing. """
        
        if self.remove_tags:
            self._remove_hashtag()
            
        if self.remove_stopwords:
            self._remove_stopwords()
            
        if self.remove_urls:
            self._remove_urls()
            
        if self.remove_mentions:
            self._remove_mentions()
        
        if self.remove_punctuation:
            self._remove_punctuation()
            
        return self.tweets

## Tweet data mining

We use the Tweepy.Cursor functionality to search for tweets on the coronavirus topic in a certain geographical area in the United States. We take New York City because this region has been most affected by the virus.

In [5]:
NUM_TWEETS = 1000
SEARCH_TERM = "#CoronaHoax -filter:retweets"

In [14]:
items = tweepy.Cursor(api.search,
            q=SEARCH_TERM, 
            geocode="40.7282,-73.7949,10000km",
            count=100,
            lang="en",
            include_rts=False,
            tweet_mode="extended").items(NUM_TWEETS)

items = list(items)
print(f"Finished reading {len(items)} items.")
denial_tweets = [t.full_text for t in items]

Finished reading 693 items.


In [7]:
covidhoax_items = tweepy.Cursor(api.search,
                    q="#CovidHoax -filter:retweets",
                    geocode="40.7282,-73.7949,1000km",
                    count=100,
                    lang="en",
                    include_rts=False,
                    tweet_mode="extended").items(NUM_TWEETS)
print(f"Finished reading {len(list(covidhoax_items))} items.")
for item in list(covidhoax_items):
    denial_tweets.append(item.full_text)

Finished reading 127 items.


In [8]:
items = tweepy.Cursor(api.search,
            q="coronavirus -filter:retweets", 
            geocode="40.7282,-73.7949,1000km",
            count=100,
            lang="en",
            include_rts=False,
            tweet_mode="extended").items(1000)

items = list(items)
print(f"Finished reading {len(items)} items.")
control_tweets = [t.full_text for t in items]

Finished reading 1000 items.


### Create a simple classification model

In [9]:
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score

In [10]:
# Do some preprocessing on the text
tweets = denial_tweets + control_tweets
preprocessor = PreProcessTweets(
                                tweets.copy(), 
                                remove_tags=True,
                                remove_urls=True,
                                remove_stopwords=True,
                                remove_mentions=True
                               )
corpus = preprocessor.preprocess()
labels = [0]*len(denial_tweets) + [1]*len(control_tweets)

In [11]:
# Vectorize the input data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    labels, 
                                                    random_state=123,
                                                    test_size=0.3
                                                   )

# Train a Naive Bayes classifier
model = MultinomialNB()
model = model.fit(X_train, y_train)

# Get some performance metrics on the training set
y_predict = model.predict(X_train)

a = accuracy_score(y_train, y_predict)
p = precision_score(y_train, y_predict)
r = recall_score(y_train, y_predict)
f = f1_score(y_train, y_predict)
print("Training performance metrics: ")
print(f"\t-Accuracy: {a:.3f},\n\t-Precision: {p:.3f}, \n\t-Recall: {r:.3f},\n\t-F1: {f:.3f}")
print("="*35)

# Get some performance metrics on the test set
y_predict = model.predict(X_test)

a = accuracy_score(y_test, y_predict)
p = precision_score(y_test, y_predict)
r = recall_score(y_test, y_predict)
f = f1_score(y_test, y_predict)
print("Test performance metrics: ")
print(f"\t-Accuracy: {a:.3f},\n\t-Precision: {p:.3f}, \n\t-Recall: {r:.3f},\n\t-F1: {f:.3f}")

Training performance metrics: 
	-Accuracy: 0.960,
	-Precision: 0.965, 
	-Recall: 0.972,
	-F1: 0.969
Test performance metrics: 
	-Accuracy: 0.794,
	-Precision: 0.881, 
	-Recall: 0.803,
	-F1: 0.840


### Real-world test

We can download some more tweets from another similar hashtag and check that these are indeed flagged correctly as "COVID denial" tweets.

In [12]:
items = tweepy.Cursor(api.search,
            q="#FakePandemic -filter:retweets", 
            geocode="40.7282,-73.7949,100km",
            count=100,
            lang="en",
            include_rts=False,
            tweet_mode="extended").items(100)

items = list(items)
print(f"Finished reading {len(items)} items.")
tweets = [t.full_text for t in items]

Finished reading 49 items.


In [13]:
x = vectorizer.transform(tweets)
y_predict = model.predict(x)
print(f"Accuracy: {list(y_predict).count(0) / len(y_predict):.3f}")

Accuracy: 0.755


### Get relevant hashtags

Starting from #CovidHoax, we collect hashtags that are commonly used in the same tweet to identify hashtags that are used by hoax believers

In [38]:
class HashtagFinder():
    
    def __init__(self, starting_hashtag):
        self.starting_hashtag = starting_hashtag
        self.hashtags = {starting_hashtag: 0}
        
    def collect_tweets(self, tag=""):
        if tag == "":
            tag = self.starting_hashtag
        items = tweepy.Cursor(api.search,
                             q = tag + " -filter:retweets",
                             geocode="40.7282,-73.7949,1000km",
                             count=100,
                             lang="en",
                             include_rts=False,
                             tweet_mode="extended").items(1000)
        items = list(items)
        print(f"Finished reading {len(items)} items.")
        tweets = [t.full_text for t in items]
        for index, tweet in enumerate(tweets):
            words = tweet.split()
            tags = [word for word in words if "#" in word]
            for tag in tags:
                if tag in self.hashtags:
                    self.hashtags[tag] = self.hashtags[tag] + 1
                else:
                    self.hashtags[tag] = 1
        
    def get_hashtag(self):
        return self.hashtags
        

In [39]:
finder = HashtagFinder("#CoronaHoax")
finder.collect_tweets()
tags = finder.get_hashtag()
print(tags)

Finished reading 552 items.
{'#CoronaHoax': 487, '#FauciTheFraud': 8, '#DemocratsAreCorrupt': 1, '#CharlieBaker': 1, '#Baker': 1, '#MaPoli': 2, '#covid': 3, '#COVIDー19': 11, '#COVID19': 61, '#COVIDIOTS': 9, '#corona': 2, '#Shiva4Senate': 1, '#3moremonths': 2, '#FireFauci': 7, '#Trump2020': 92, '#Coronahoax': 16, '#AuditHoax': 1, '#MexicoWillPayForTheWallHoax': 1, '#RussiaHoax': 2, '#PerfectCallHoax.': 1, '#EmptyHospitals': 1, '#DancingNurses': 1, '#BadDoctors': 1, '#CovidHaox': 1, '#ObamaGate': 12, '#Corruption': 2, '#coronahoax': 33, '#QSentMe': 2, '#TheGreatAwakening': 4, '#TheMoreYouKnow': 1, '#WWG1WGAWORLDWIDE': 4, '#': 4, '#coronavirus': 109, '#CornavirusOutbreak': 1, '#MAGA': 9, '#WakeUpAmerica': 9, '#WakeUpCalifornia': 1, '#WakeUpWorld': 3, '#Lockdown': 1, '#EndTheShutdownNow': 1, '#NoMasks': 4, '#OpenTheEconomy': 1, '#IDIOTS': 1, '#GMO': 1, '#vaccines': 2, '#VaccineInjury': 1, '#CoronavirusLiar': 12, '#WakeUp': 2, '#sheeple': 1, '#truth': 1, '#discernment': 1, '#takingyoursoul'